In [1]:
import sys
import random
from typing import List, Optional

def compute_c_from_zeros(zeros: List[int]) -> int:
    """Compute c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{zeros[j]} for sorted zeros list."""
    if not zeros:
        return 0
    b = len(zeros)
    return sum((3 ** (b - j - 1)) * (1 << zeros[j]) for j in range(b))

def zeros_from_abc(a: int, b: int, c: int, tail_ones: int = 0,
                   max_recursions: int = 10_000_000) -> Optional[List[int]]:
    """
    Recover a strictly increasing list of zero indices Z = [i0, ..., i_{b-1}] with
      0 <= i0 < i1 < ... < i_{b-1} <= a-1-tail_ones
    such that
      c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{i_j}.
    Parameters:
      - a: total bitstring length
      - b: required number of zeros
      - c: target integer
      - tail_ones: number of trailing forced '1' bits (so zeros cannot be in last tail_ones positions)
      - max_recursions: safety cutoff for recursion count (to avoid pathological long runs)
    Returns:
      - list of indices (length b) if found, otherwise None.
    """
    # Quick checks
    if a <= 0 or b < 0 or c < 0:
        return None
    if b == 0:
        return [] if c == 0 else None
    if b > a - tail_ones:
        # Not enough positions to place b zeros
        return None

    # Precompute powers of two up to a-1-tail_ones
    max_index = a - 1 - tail_ones
    if max_index < 0:
        return None
    pow2 = [1 << i for i in range(max_index + 1)]

    # recursion counter for safety
    rec_counter = 0

    # helper to compute minimal and maximal possible contribution for remaining k slots
    # given next index must be >= min_i and <= max_i
    def bound_min_max(k: int, min_i: int, max_i: int) -> (int, int):
        """
        For k remaining positions, with allowed index range [min_i, max_i] (inclusive),
        compute the minimal and maximal possible values of
           S = sum_{t=0}^{k-1} 3^{k-1-t} * 2^{z_t}
        where min_i <= z_0 < z_1 < ... < z_{k-1} <= max_i.
        Minimal S: choose z_t = min_i + t (smallest increasing)
        Maximal S: choose z_t = max_i - (k-1 - t) (largest increasing)
        """
        if k <= 0:
            return 0, 0
        # minimal indices
        zmin = [min_i + t for t in range(k)]
        zmax = [max_i - (k - 1 - t) for t in range(k)]
        # compute Smin and Smax
        Smin = sum((3 ** (k - 1 - t)) * pow2[zmin[t]] for t in range(k))
        Smax = sum((3 ** (k - 1 - t)) * pow2[zmax[t]] for t in range(k))
        return Smin, Smax

    # backtracking with pruning
    solution = None

    def backtrack(pos: int, remaining: int, last_index: int):
        # pos: how many zeros chosen so far (we fill left->right)
        # remaining: remaining value of c to satisfy for positions pos..b-1
        # last_index: last chosen index (we must pick next > last_index)
        nonlocal rec_counter, solution
        rec_counter += 1
        if rec_counter > max_recursions:
            return

        if pos == b:
            # all chosen; success if remaining == 0
            if remaining == 0:
                return []  # empty tail indicates success (build on unwind)
            else:
                return None

        # weight for the current position (pos corresponds to coefficient 3^{b-1-pos})
        weight = 3 ** (b - 1 - pos)

        # compute bounds for remaining k positions including this one
        min_next_index = last_index + 1
        max_next_index = max_index - (b - 1 - pos)  # ensure space for remaining
        if min_next_index > max_next_index:
            return None

        k_remaining = b - pos
        # quick pruning: compute min and max possible sums for all k_remaining positions
        Smin, Smax = bound_min_max(k_remaining, min_next_index, max_index)
        if remaining < Smin or remaining > Smax:
            return None

        # We must pick i in [min_next_index, max_next_index] such that term = weight * 2**i <= remaining
        # iterate i from min_next_index upward (small indices first tends to produce lexicographically small solution)
        # but we can also iterate from max to min if prefer larger indices first
        # To cut branches, compute max_i_allowed_by_remaining = floor(log2(remaining/weight))
        # but ensure within index bounds.
        if weight == 0:
            return None
        max_i_by_rem = -1
        # remaining/weight might be huge; use integer division to find limit
        limit = remaining // weight
        # if limit < 1 then no term possible
        if limit < 1:
            return None
        # find largest i with 2^i <= limit but also <= max_next_index + (b-1-pos) ??? we already set max_next_index
        # safe compute:
        # binary search/power lookup on pow2
        lo, hi = 0, max_index
        while lo <= hi:
            mid = (lo + hi) // 2
            if pow2[mid] <= limit:
                max_i_by_rem = mid
                lo = mid + 1
            else:
                hi = mid - 1
        # effective upper bound for i
        i_upper = min(max_next_index, max_i_by_rem)
        if i_upper < min_next_index:
            return None

        # iterate candidate indices i in sensible order: we try smaller indices first (to favor compact spacing)
        for i in range(min_next_index, i_upper + 1):
            term = weight * pow2[i]
            if term > remaining:
                break
            # pruning for the suffix after choosing i:
            rem_after = remaining - term
            # for the remaining k_remaining-1 positions, allowed index range is [i+1, max_index]
            if k_remaining - 1 > 0:
                Smin_tail, Smax_tail = bound_min_max(k_remaining - 1, i + 1, max_index)
                if rem_after < Smin_tail or rem_after > Smax_tail:
                    continue
            # recursive descend
            res_tail = backtrack(pos + 1, rem_after, i)
            if res_tail is not None:
                return [i] + res_tail
        # no candidate worked
        return None

    result = backtrack(0, c, -1)
    return result


In [2]:
zeros_from_abc(18, 5, 24976)

[4, 7, 8, 9, 14]

In [3]:
zeros_from_abc(14, 5, 1804)

[2, 3, 4, 5, 10]

In [4]:
def s_from_abc(a, b, c):
    z_idxs = zeros_from_abc(a, b, c)
    if z_idxs is None:
        print("zeros_from_abc failed!")
        print((a,b,c))
        return ""
    V = [1]*a
    for idx in z_idxs:
        if idx >= len(V):
            print("Index overflow")
            print((idx, (a, b, c), z_idxs))
        else:
            V[idx] = 0
    return "".join(map(str, V))
#

In [5]:
s_from_abc(18, 5, 24976)

'111101100011110111'

In [6]:
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#


In [7]:
def Z(s):
    for i in range(len(s)):
        if s[i] == "0":
            yield i
#

In [8]:
def a_b_c(s):
    a = len(s)
    b = 0
    for bit in s:
        if bit == "0":
            b += 1
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(Z(s)))
    return (a,b,c)

In [9]:
for i in range(300):
    x = random.randint(20, 1000)
    print(x)
    sys.stdout.flush()
    s = collatzPath(x)
    a, b, c = a_b_c(s)
    ss = s_from_abc(a, b, c)
    if s != ss:
        print((x, s, ss))
#

701
zeros_from_abc failed!
(54, 28, 1977766998554323)
(701, '011000101100010010000001100001110101011101100011110111', '')
286
zeros_from_abc failed!
(67, 37, 18792755504851167110)
(286, '1000011100100010000101100010010000001100001110101011101100011110111', '')
759
zeros_from_abc failed!
(40, 19, 217355174323)
(759, '0001101010100001111100101011100010110111', '')
565
503
zeros_from_abc failed!
(44, 22, 1807513061089)
(503, '00011010000001100001110101011101100011110111', '')
279
630
491
zeros_from_abc failed!
(90, 51, 180476303415281411640547447)
(491, '001010000010010101111011000010100100010000101100010010000001100001110101011101100011110111', '')
211
286
zeros_from_abc failed!
(67, 37, 18792755504851167110)
(286, '1000011100100010000101100010010000001100001110101011101100011110111', '')
780
zeros_from_abc failed!
(78, 43, 46191020335918373127484)
(780, '110011100011000010100100010000101100010010000001100001110101011101100011110111', '')
91
zeros_from_abc failed!
(59, 33, 70586240746870

In [ ]:
zeros_from_abc(74, 41, 2513090558128897759837)

In [47]:
336, a_b_c(collatzPath(336))

(336, (10, 1, 16))

In [48]:
zeros_from_abc(10, 1, 16)

[4]

In [27]:
collatzPath(426), a_b_c(collatzPath(426))

('101111110111', (12, 2, 262))

In [28]:
zeros_from_abc(12, 2, 262)

[1, 8]

In [29]:
indexes = [1, 8]
S = ['1']*12
for idx in indexes:
    S[idx] = "0"



In [30]:
S

['1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1']